In [27]:
from jazzstock_bot.common import connector_db as db
import pandas as pd
import time
import sys


pd.options.display.max_rows = 1000
pd.options.display.max_columns= 500
pd.options.display.expand_frame_repr=False

def db_readAll():
    # DB에서 [종목명,종목코드] 로 구성된 데이터셋을 받아옴.
    # dbUpdateDate = db.selectSingleValue('SELECT max(date) FROM test.t_stock_shares_info')

    query = """

                        SELECT A.STOCKCODE, A.STOCKNAME
                        FROM jazzdb.T_STOCK_CODE_MGMT A
                        WHERE 1=1
                        AND A.LISTED = 1
                                                        """

    for eachRow in db.select(query):
        if (len(eachRow) > 0):
            itemDic[eachRow[1].upper()] = eachRow[0]
            codeDic[eachRow[0]] = eachRow[1].upper()

    print("[INFO] 종목명/종목코드를 메모리에 읽어왔습니다, 남은 종목 수: ", len(itemDic.keys()))
    
    
class StockMacd:
    
    
    def __init__(self, stockcode):
        
        self.stockcode = stockcode
        self.df = self.get_ohlc()
        self._macd()
        
    def get_ohlc(self, cnt=900):
        
        date_from = db.selectSingleValue('SELECT CAST(DATE AS CHAR) AS DATE FROM jazzdb.T_DATE_INDEXED WHERE CNT = "%s"'%(cnt))
        df = db.selectpd("SELECT STOCKCODE, CAST(DATE AS CHAR) AS DATE, A.OPEN, A.HIGH, A.LOW, A.CLOSE, VOLUME FROM jazzdb.T_STOCK_OHLC_DAY A JOIN jazzdb.T_STOCK_SND_DAY B USING(STOCKCODE, DATE) WHERE STOCKCODE = '%s' AND DATE > '%s'"%(self.stockcode, date_from))
        return df


    def _macd(self, fast=12, slow=26, signal=9):
        self.df['MACD'] = self.df['CLOSE'].ewm(span=fast, adjust=False).mean() - self.df['CLOSE'].ewm(span=slow, adjust=False).mean()
        self.df['SIGNAL'] = self.df['MACD'].ewm(span=signal, min_periods = signal-1).mean()
        self.df['MACD_OSC'] = self.df['MACD'] - self.df['SIGNAL']
    

    def gettoday():
        td = db.selectSingleValue('SELECT cast(DATE AS CHAR) AS DATE FROM jazzdb.T_DATE_INDEXED WHERE CNT = 0')
        return td
    
    def get_dataframe(self):
        return self.df
    
    
    def insert_to_database(self, n=3):
        db.insertdf(self.df[['STOCKCODE', 'DATE', 'PSMAR5','PSMAR20', 'PSMAR60', 'PSMAR120', 'PSMAR240', 'VSMAR5','VSMAR20','VSMAR60', 'VSMAR120', 'VSMAR240']].round(3).tail(n), table='jazzdb.T_STOCK_DAY_SMAR')

        


In [32]:
print(StockMacd('093320').df[['DATE', 'CLOSE','MACD', 'SIGNAL', 'MACD_OSC']].tail(40))

           DATE  CLOSE         MACD       SIGNAL    MACD_OSC
851  2021-04-28  69400 -1536.346883 -1261.728540 -274.618343
852  2021-04-29  69300 -1609.065920 -1331.196016 -277.869904
853  2021-04-30  69200 -1655.679800 -1396.092773 -259.587027
854  2021-05-03  69100 -1681.309707 -1453.136160 -228.173548
855  2021-05-04  69000 -1690.207103 -1500.550348 -189.656755
856  2021-05-06  68900 -1685.893585 -1537.618996 -148.274590
857  2021-05-07  69700 -1599.483922 -1549.991981  -49.491941
858  2021-05-10  70200 -1473.670267 -1534.727638   61.057371
859  2021-05-11  69600 -1406.167600 -1509.015630  102.848031
860  2021-05-12  66500 -1584.549640 -1524.122432  -60.427207
861  2021-05-13  65500 -1786.022182 -1576.502382 -209.519800
862  2021-05-14  65600 -1915.540303 -1644.309967 -271.230337
863  2021-05-17  64200 -2106.866007 -1736.821175 -370.044833
864  2021-05-18  62100 -2400.276562 -1869.512252 -530.764310
865  2021-05-20  61400 -2658.643502 -2027.338502 -631.305000
866  2021-05-21  61900 -

In [5]:
codeDic, itemDic = {}, {}
db_readAll()

[INFO] 종목명/종목코드를 메모리에 읽어왔습니다, 남은 종목 수:  2546


In [7]:
for i, stockcode in enumerate(codeDic.keys()): 
    StockMovingAverage(stockcode).insert_to_database(n=1)
    print(i, stockcode)

0 000020
1 000040
2 000050
3 000060
4 000070
5 000075
6 000080
7 000087
8 000100
9 000105
10 000120
11 000140
12 000145
13 000150
14 000155
15 000157
16 000180
17 000210
18 000215
19 000220
20 000225
21 000227
22 000230
23 000240
24 000250
25 000270
26 000300
27 000320
28 000325
29 000370
30 000390
31 000400
32 000430
33 000440
34 000480
35 000490
36 000500
37 000520
38 000540
39 000545
40 000547
41 000590
42 000640
43 000650
44 000660
45 000670
46 000680
47 000700
48 000720
49 000725
50 000760
51 000810
52 000815
53 000850
54 000860
55 000880
56 000885
57 00088K
58 000890
59 000910
60 000950
61 000970
62 000990
63 000995
64 001000
65 001020
66 001040
67 001045
68 001060
69 001065
70 001067
71 001070
72 001080
73 001120
74 001130
75 001140
76 001200
77 001210
78 001230
79 001250
80 001260
81 001270
82 001275
83 001290
84 001340
85 001360
86 001380
87 001390
88 001420
89 001430
90 001440
91 001450
92 001460
93 001465
94 001470
95 001500
96 001510
97 001515
98 001520
99 001525
100 001527

In [52]:
df = db.selectpd('''

SELECT STOCKNAME, PSMAR5, PSMAR20, PSMAR60, VSMAR5, VSMAR20, VSMAR60
FROM jazzdb.T_STOCK_DAY_SMAR
JOIN jazzdb.T_STOCK_CODE_MGMT USING (STOCKCODE)
WHERE 1=1
AND DATE = '2021-03-05';

''')


df.to_csv("sma.csv", index=False)